# notebook for generating lowTh events

This is a roundabout solution that *will be changed*.

Start with a bunch of the fun IC packages that you want (fit this to your flavour of IC)

In [2]:
import sys,os,os.path
# sys.path.append("../../../") # if you move files around, you need to adjust this!
# sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
# os.environ['ICTDIR']='/home/e78368jw/Documents/NEXT_CODE/IC'

# os.environ["ICTDIR"] = "~/Packages/IC"
# os.environ["ICDIR"] = "$ICTDIR/invisible_cities"


#%load_ext autoreload
#%autoreload 2
#%matplotlib notebook

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['mathtext.fontset'] = 'stix'
rcParams['font.family'] = 'STIXGeneral'
rcParams['figure.figsize'] = [10, 8]
rcParams['font.size'] = 22

import pandas as pd
import numpy  as np
import tables as tb

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as clrs

import invisible_cities.core.core_functions                   as     coref
import invisible_cities.io.dst_io                           as     dstio

from invisible_cities.cities                 import beersheba as beerfun

from invisible_cities.evm.event_model                          import HitCollection

from invisible_cities.database.load_db       import DataSiPM

from invisible_cities.evm.event_model        import Cluster, Hit
from invisible_cities.types.ic_types         import xy
from invisible_cities.reco.paolina_functions import voxelize_hits

from invisible_cities.evm.event_model        import HitEnergy
from invisible_cities.cities.beersheba          import DeconvolutionMode
from invisible_cities.cities.beersheba          import CutType

from invisible_cities.reco import hits_functions as hif

from invisible_cities.reco.deconv_functions import deconvolve
from invisible_cities.reco.deconv_functions import deconvolution_input
from invisible_cities.reco.deconv_functions import InterpolationMethod

import invisible_cities.io.mcinfo_io as mcio

from collections import defaultdict

import matplotlib.cm as cm
from matplotlib.colors import Normalize



### Pull out a couple of functions that may be deprecated, i can't remember

But they're needed for lowTh creation, and I wanted them in this file to better understand what I'm doing.

In [3]:


def threshold_hits(threshold_charge, same_peak, hitc):
    """
    Applies a threshold to hits and redistributes the charge/energy.

    Parameters
    ----------
    threshold_charge : float
        minimum pes of a hit
    same_peak        : bool
        whether to reassign NN hits' energy only to the hits from the same peak

    Returns
    ----------
    A function that takes HitCollection as input and returns another object with
    only non NN hits of charge above threshold_charge.
    The energy of NN hits is redistributed among neighbors.
    """

    t = hitc.time
    thr_hits = hif.threshold_hits(hitc.hits, threshold_charge     )
    mrg_hits = hif.merge_NN_hits ( thr_hits, same_peak = same_peak)

    cor_hits = []
    for hit in mrg_hits:
        cluster = Cluster(hit.Q, xy(hit.X, hit.Y), hit.var, hit.nsipm)
        xypos   = xy(hit.Xpeak, hit.Ypeak)
        hit     = Hit(hit.npeak, cluster, hit.Z, hit.E, xypos, hit.Ec)
        cor_hits.append(hit)

    new_hitc      = HitCollection(hitc.event, t)
    new_hitc.hits = cor_hits
    return new_hitc





def hits_from_df (dst, skip_NN = False):
    """
    Function that transforms pandas DataFrame dst to HitCollection
    ------
    Parameters
    ------
    dst : pd.DataFrame
        DataFrame with obligatory columns :
                event, npeak, X, Y, Z,  Q, E
        If time, nsipm, Xrms, Yrms, Qc, Ec, track_id are not
        inside dst the default value is set to -1
        If Xpeak, Ypeak not in dst the default value is -1000
    ------
    Returns
    ------
    Dictionary {event_number : HitCollection}
    from here
    https://github.com/next-exp/IC/blob/v2-development/invisible_cities/io/hits_io.py#L16
    """
    all_events = {}
    times = getattr(dst, 'time', [-1]*len(dst))
    for (event, time) , df in dst.groupby(['event', times]):
        #pandas is not consistent with numpy dtypes so we have to change it by hand
        event = np.int32(event)
        hits  = []
        for i, row in df.iterrows():
            Q = getattr(row,'Q', row.E)
            if skip_NN and Q == NN:
                continue
            if hasattr(row, 'Xrms'):
                Xrms  = row.Xrms
                Xrms2 = Xrms**2
            else:
                Xrms = Xrms2 = -1
            if hasattr(row, 'Yrms'):
                Yrms  = row.Yrms
                Yrms2 = Yrms**2
            else:
                Yrms = Yrms2 = -1
            nsipm   = getattr(row, 'nsipm'   , -1   )     # for backwards compatibility
            Qc      = getattr(row, 'Qc'      , -1   )     # for backwards compatibility
            Xpeak   = getattr(row, 'Xpeak'   , -1000)     # for backwards compatibility
            Ypeak   = getattr(row, 'Ypeak'   , -1000)     # for backwards compatibility
            Ec      = getattr(row, 'Ec'      , -1   )     # for backwards compatibility
            trackID = getattr(row, 'track_id', -1   )     # for backwards compatibility
            Ep      = getattr(row, "Ep"      , -1   )     # for backwards compatibility

            hit = Hit(row.npeak            ,
                      Cluster(Q               ,
                              xy(row.X, row.Y),
                              xy(Xrms2, Yrms2),
                              nsipm = nsipm   ,
                              z     = row.Z   ,
                              E     = row.E   ,
                              Qc    = Qc      ),
                      row.Z                ,
                      row.E                ,
                      xy(Xpeak, Ypeak)     ,
                      s2_energy_c = Ec     ,
                      track_id    = trackID,
                      Ep          = Ep     )

            hits.append(hit)

        if len(hits):
            all_events[event] = HitCollection(event, time, hits=hits)

    return all_events

def hitc_to_df_(hitc):
    columns = defaultdict(list)
    for hit in hitc.hits:
        columns["event"   ].append(hitc.event)
        columns["time"    ].append(hitc.time)
        columns["npeak"   ].append(hit .npeak)
        columns["Xpeak"   ].append(hit .Xpeak)
        columns["Ypeak"   ].append(hit .Ypeak)
        columns["nsipm"   ].append(hit .nsipm)
        columns["X"       ].append(hit .X)
        columns["Y"       ].append(hit .Y)
        columns["Xrms"    ].append(hit .Xrms)
        columns["Yrms"    ].append(hit .Yrms)
        columns["Z"       ].append(hit .Z)
        columns["Q"       ].append(hit .Q)
        columns["E"       ].append(hit .E)
        columns["Qc"      ].append(hit .Qc)
        columns["Ec"      ].append(hit .Ec)
        columns["track_id"].append(hit .track_id)
        columns["Ep"      ].append(hit .Ep)
    return pd.DataFrame(columns)


def soph_to_lowTh(df, threshold = 5):
    '''
    Converts sophronia 'RECO/Events' to lowTh events via a rather convoluted process

    ------
    Parameters
    ------
    df : pd.DataFrame
        DataFrame with obligatory columns :
                event, npeak, X, Y, Z,  Q, E
    threshold: int
        value at which the threshold is set.
    ------
    Returns
    ------
    Dictionary {event_number : HitCollection}
    from here
    '''

    # safety check, to ensure you don't accidentally make a repeating dataframe
    


    # new parameters for threshold, this is silly but I'm copying previous convention
    pes = 1
    threshold = threshold * pes
    same_peak = True

    # convert sophronia RECO/Events to hit collection
    soph_hitc = hits_from_df(df)

    # collect the keys as the event numbers
    soph_hitc_list = list(soph_hitc.keys())

    print("Processing data...")
    # loop over all of these events
    j = 0
    for i in soph_hitc_list:
        j += 1

        if (len(soph_hitc_list)%j == 50): 
            print("{}/{}".format(j, len(soph_hitc_list)))
        # choose i'th event
        soph_hit_event = soph_hitc.get(i)

        # Apply threshold calculation
        soph_hitc_lowTh = threshold_hits(threshold, same_peak, soph_hit_event)

        # convert back to pandas dataframe using hitc_to_df
        soph_hdst_lowTh = hitc_to_df_(soph_hitc_lowTh)

        # check if pandas dataframe with all the events exists yet
        if 'full_soph_df' in locals() and isinstance(full_soph_df, pd.DataFrame):
            full_soph_df = pd.concat([full_soph_df, soph_hdst_lowTh])
        else:
            full_soph_df = soph_hdst_lowTh.copy(deep = True)
    
    return full_soph_df

        


### EXAMPLE CASE: using 1 sophronia file

In [4]:
# loading sophronia file
soph_file = f'../data/nexus/LPR_Tl208_Ports/PORT_1a/sophronia/sophronia_116_208Tl.h5'
soph_hdst = dstio.load_dst(soph_file, 'RECO', 'Events')#CHITS/lowTh
display(soph_hdst)

,event,time,npeak,Xpeak,Ypeak,nsipm,X,Y,Xrms,Yrms,Z,Q,E,Qc,Ec,track_id,Ep
0,2300000,2.300002e+12,0,-191.881857,304.954168,1,-178.825,287.675,0.0,0.0,40.536556,7.130435,374.585728,-1.0,0.001169,-1,-1.0
1,2300000,2.300002e+12,0,-191.881857,304.954168,1,-178.825,303.225,0.0,0.0,40.536556,20.753624,1090.257673,-1.0,0.003422,-1,-1.0
2,2300000,2.300002e+12,0,-191.881857,304.954168,1,-178.825,318.775,0.0,0.0,40.536556,12.579710,660.854468,-1.0,0.002075,-1,-1.0
3,2300000,2.300002e+12,0,-191.881857,304.954168,1,-163.275,303.225,0.0,0.0,40.536556,12.289855,645.627432,-1.0,0.002027,-1,-1.0
4,2300000,2.300002e+12,0,-191.881857,304.954168,1,-163.275,318.775,0.0,0.0,40.536556,9.797101,514.674698,-1.0,0.001616,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173737,2300676,2.300676e+12,0,182.684460,395.165054,1,178.825,396.525,0.0,0.0,210.447784,13.391304,500.220978,-1.0,0.001719,-1,-1.0
173738,2300676,2.300676e+12,0,182.684460,395.165054,1,194.375,380.975,0.0,0.0,210.447784,7.014493,262.020538,-1.0,0.000911,-1,-1.0
173739,2300676,2.300676e+12,0,182.684460,395.165054,1,194.375,396.525,0.0,0.0,210.447784,10.608696,396.279022,-1.0,0.001378,-1,-1.0
173740,2300676,2.300676e+12,0,182.684460,395.165054,1,194.375,412.075,0.0,0.0,210.447784,9.913043,370.293457,-1.0,0.001349,-1,-1.0


In [5]:
# convert to hitcollection
soph_hits = hits_from_df(soph_hdst)

## Demonstrating for 1 event:

In [6]:
# set the event that you want 
event = 1

soph_hits_list = list(soph_hits.keys())
event_soph_hits = soph_hits_list[event]

print(event_soph_hits)

soph_hit_event = soph_hits.get(event_soph_hits)

#print(soph_hit_event)

2300002


### Here is where you define the threshold

lowTh being 5 * pes here

In [7]:
# new parameters for threshold
pes = 1
threshold = 5 * pes
same_peak = True

# Apply threshold calculation
soph_hitc_lowTh = threshold_hits(threshold, same_peak, soph_hit_event)

### convert back to pandas dataframe using hitc_to_df

In [8]:
# convert back to pandas dataframe using hitc_to_df
soph_hdst_lowTh = hitc_to_df_(soph_hitc_lowTh)

In [9]:
display(soph_hdst_lowTh)

,event,time,npeak,Xpeak,Ypeak,nsipm,X,Y,Xrms,Yrms,Z,Q,E,Qc,Ec,track_id,Ep
0,2300002,2.300002e+12,0.0,143.088964,282.720789,1.0,147.725,287.675,0.0,0.0,377.522278,5.101449,2113.000000,-1,0.006802,-1,-1
1,2300002,2.300002e+12,0.0,143.088964,282.720789,1.0,209.925,225.475,0.0,0.0,378.560577,5.855072,435.467345,-1,0.001400,-1,-1
2,2300002,2.300002e+12,0.0,143.088964,282.720789,1.0,132.175,287.675,0.0,0.0,378.560577,6.028986,448.402016,-1,0.001450,-1,-1
3,2300002,2.300002e+12,0.0,143.088964,282.720789,1.0,147.725,272.125,0.0,0.0,378.560577,6.956522,517.386925,-1,0.001666,-1,-1
4,2300002,2.300002e+12,0.0,143.088964,282.720789,1.0,147.725,287.675,0.0,0.0,378.560577,10.724638,797.638225,-1,0.002568,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,2300002,2.300002e+12,0.0,143.088964,282.720789,1.0,132.175,303.225,0.0,0.0,401.576508,6.956522,334.221126,-1,0.001090,-1,-1
215,2300002,2.300002e+12,0.0,143.088964,282.720789,1.0,147.725,287.675,0.0,0.0,401.576508,6.956522,334.221126,-1,0.001078,-1,-1
216,2300002,2.300002e+12,0.0,143.088964,282.720789,1.0,116.625,287.675,0.0,0.0,402.616150,11.768116,893.450684,-1,0.002878,-1,-1
217,2300002,2.300002e+12,0.0,143.088964,282.720789,1.0,116.625,303.225,0.0,0.0,402.616150,7.014493,532.549438,-1,0.001726,-1,-1


### This can be done for a full dataframe rather than one event

With this fun function I made :)

Be careful it takes a while.

In [10]:
soph_lowTh = soph_to_lowTh(soph_hdst, threshold = 5)
display(soph_lowTh)

Processing data...
281/331


,event,time,npeak,Xpeak,Ypeak,nsipm,X,Y,Xrms,Yrms,Z,Q,E,Qc,Ec,track_id,Ep
0,2300000,2.300002e+12,0.0,-191.881857,304.954168,1.0,-178.825,287.675,0.0,0.0,40.536556,7.130435,374.585736,-1,0.001169,-1,-1
1,2300000,2.300002e+12,0.0,-191.881857,304.954168,1.0,-178.825,303.225,0.0,0.0,40.536556,20.753624,1090.257681,-1,0.003422,-1,-1
2,2300000,2.300002e+12,0.0,-191.881857,304.954168,1.0,-178.825,318.775,0.0,0.0,40.536556,12.579710,660.854484,-1,0.002075,-1,-1
3,2300000,2.300002e+12,0.0,-191.881857,304.954168,1.0,-163.275,303.225,0.0,0.0,40.536556,12.289855,645.627426,-1,0.002027,-1,-1
4,2300000,2.300002e+12,0.0,-191.881857,304.954168,1.0,-163.275,318.775,0.0,0.0,40.536556,9.797101,514.674673,-1,0.001616,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2300676,2.300676e+12,0.0,182.684460,395.165054,1.0,178.825,396.525,0.0,0.0,210.447784,13.391304,500.221002,-1,0.001719,-1,-1
196,2300676,2.300676e+12,0.0,182.684460,395.165054,1.0,194.375,380.975,0.0,0.0,210.447784,7.014493,262.020540,-1,0.000911,-1,-1
197,2300676,2.300676e+12,0.0,182.684460,395.165054,1.0,194.375,396.525,0.0,0.0,210.447784,10.608696,396.278998,-1,0.001378,-1,-1
198,2300676,2.300676e+12,0.0,182.684460,395.165054,1.0,194.375,412.075,0.0,0.0,210.447784,9.913043,370.293461,-1,0.001349,-1,-1
